# `pandas` - Summarization and Grouping

__Contents__: 
1. Setup
1. Grouping data
1. Aggregation with `agg()` method
1. Other aggregating functions

## Reference
- http://pandas.pydata.org/pandas-docs/stable/index.html
- https://pandas.pydata.org/pandas-docs/stable/dsintro.html
- https://pandas.pydata.org/pandas-docs/stable/groupby.html

## 1. Setup

Load the libraries.

In [1]:
import pandas  as pd
import numpy   as np
(pd.__version__,
 np.__version__
)

('0.24.2', '1.16.4')

Load the DataFrame from the `imports-85.csv` CSV file. Set the column names.

In [2]:
%%sh
git clone https://github.com/datalab-datasets/file-samples.git

Cloning into 'file-samples'...


In [3]:
%ls /content/file-samples/imports-85.csv

/content/file-samples/imports-85.csv


In [0]:
column_names = ['symboling', 'normalized-losses', 'make', 'fuel-type',
                'aspiration', 'num-of-doors', 'body-style', 'drive-wheels',
                'engine-location', 'wheel-base', 'length', 'width',
                'height', 'curb-weight', 'engine-type', 'num-of-cylinders',
                'engine-size', 'fuel-system', 'bore', 'stroke',
                'compression-ratio', 'horsepower', 'peak-rpm', 'city-mpg',
                'highway-mpg', 'price']
import_df = pd.read_csv('/content/file-samples/imports-85.csv',
                        names=[string.replace('-','_') for string in column_names],
                        na_values=['?']
                       )

In [5]:
import_df.head()

,symboling,normalized_losses,make,fuel_type,aspiration,num_of_doors,body_style,drive_wheels,engine_location,wheel_base,length,width,height,curb_weight,engine_type,num_of_cylinders,engine_size,fuel_system,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price
0,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,13495.0
1,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,16500.0
2,1,NaN,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154.0,5000.0,19,26,16500.0
3,2,164.0,audi,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102.0,5500.0,24,30,13950.0
4,2,164.0,audi,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115.0,5500.0,18,22,17450.0


Display basic information about each column of the DataFrame.

In [6]:
import_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
symboling            205 non-null int64
normalized_losses    164 non-null float64
make                 205 non-null object
fuel_type            205 non-null object
aspiration           205 non-null object
num_of_doors         203 non-null object
body_style           205 non-null object
drive_wheels         205 non-null object
engine_location      205 non-null object
wheel_base           205 non-null float64
length               205 non-null float64
width                205 non-null float64
height               205 non-null float64
curb_weight          205 non-null int64
engine_type          205 non-null object
num_of_cylinders     205 non-null object
engine_size          205 non-null int64
fuel_system          205 non-null object
bore                 201 non-null float64
stroke               201 non-null float64
compression_ratio    205 non-null float64
horsepower           203 non-

## 2. Grouping data

Pandas objects can be split into groups on any of their axes. To create a GroupBy object (more on what the GroupBy object later), you may do the following:
- `grouped = obj.groupby(key)`
- `grouped = obj.groupby(key, axis=1) (default is axis=0)`
- `grouped = obj.groupby([key1, key2])`

Note that `obj` is a pandas DataFrame object.

Create a GroupBy object by calling `groupby()` method of the dataframe. Below group by the `make` column.

In [7]:
grouped = import_df.groupby('make')
grouped

A single group can be selected using `get_group()`. The following code cell displays all the records that meet the condition of `make='toyota'` in the dataframe. Recall that `make` was specified above as the "group by" variable.

In [8]:
grouped.get_group('toyota')

,symboling,normalized_losses,make,fuel_type,aspiration,num_of_doors,body_style,drive_wheels,engine_location,wheel_base,length,width,height,curb_weight,engine_type,num_of_cylinders,engine_size,fuel_system,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price
150,1,87.0,toyota,gas,std,two,hatchback,fwd,front,95.7,158.7,63.6,54.5,1985,ohc,four,92,2bbl,3.05,3.03,9.0,62.0,4800.0,35,39,5348.0
151,1,87.0,toyota,gas,std,two,hatchback,fwd,front,95.7,158.7,63.6,54.5,2040,ohc,four,92,2bbl,3.05,3.03,9.0,62.0,4800.0,31,38,6338.0
152,1,74.0,toyota,gas,std,four,hatchback,fwd,front,95.7,158.7,63.6,54.5,2015,ohc,four,92,2bbl,3.05,3.03,9.0,62.0,4800.0,31,38,6488.0
153,0,77.0,toyota,gas,std,four,wagon,fwd,front,95.7,169.7,63.6,59.1,2280,ohc,four,92,2bbl,3.05,3.03,9.0,62.0,4800.0,31,37,6918.0
154,0,81.0,toyota,gas,std,four,wagon,4wd,front,95.7,169.7,63.6,59.1,2290,ohc,four,92,2bbl,3.05,3.03,9.0,62.0,4800.0,27,32,7898.0
155,0,91.0,toyota,gas,std,four,wagon,4wd,front,95.7,169.7,63.6,59.1,3110,ohc,four,92,2bbl,3.05,3.03,9.0,62.0,4800.0,27,32,8778.0
156,0,91.0,toyota,gas,std,four,sedan,fwd,front,95.7,166.3,64.4,53.0,2081,ohc,four,98,2bbl,3.19,3.03,9.0,70.0,4800.0,30,37,6938.0
157,0,91.0,toyota,gas,std,four,hatchback,fwd,front,95.7,166.3,64.4,52.8,2109,ohc,four,98,2bbl,3.19,3.03,9.0,70.0,4800.0,30,37,7198.0
158,0,91.0,toyota,diesel,std,four,sedan,fwd,front,95.7,166.3,64.4,53.0,2275,ohc,four,110,idi,3.27,3.35,22.5,56.0,4500.0,34,36,7898.0
159,0,91.0,toyota,diesel,std,four,hatchback,fwd,front,95.7,166.3,64.4,52.8,2275,ohc,four,110,idi,3.27,3.35,22.5,56.0,4500.0,38,47,7788.0


The code cell below groups by multiple columns and displays records that meet the conditions of `make='toyota'` and `body_style='sedan'`.

In [9]:
import_df.groupby(['make','body_style']).get_group(('toyota','sedan'))

,symboling,normalized_losses,make,fuel_type,aspiration,num_of_doors,body_style,drive_wheels,engine_location,wheel_base,length,width,height,curb_weight,engine_type,num_of_cylinders,engine_size,fuel_system,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price
156,0,91.0,toyota,gas,std,four,sedan,fwd,front,95.7,166.3,64.4,53.0,2081,ohc,four,98,2bbl,3.19,3.03,9.0,70.0,4800.0,30,37,6938.0
158,0,91.0,toyota,diesel,std,four,sedan,fwd,front,95.7,166.3,64.4,53.0,2275,ohc,four,110,idi,3.27,3.35,22.5,56.0,4500.0,34,36,7898.0
160,0,91.0,toyota,gas,std,four,sedan,fwd,front,95.7,166.3,64.4,53.0,2094,ohc,four,98,2bbl,3.19,3.03,9.0,70.0,4800.0,38,47,7738.0
162,0,91.0,toyota,gas,std,four,sedan,fwd,front,95.7,166.3,64.4,52.8,2140,ohc,four,98,2bbl,3.19,3.03,9.0,70.0,4800.0,28,34,9258.0
163,1,168.0,toyota,gas,std,two,sedan,rwd,front,94.5,168.7,64.0,52.6,2169,ohc,four,98,2bbl,3.19,3.03,9.0,70.0,4800.0,29,34,8058.0
165,1,168.0,toyota,gas,std,two,sedan,rwd,front,94.5,168.7,64.0,52.6,2265,dohc,four,98,mpfi,3.24,3.08,9.4,112.0,6600.0,26,29,9298.0
173,-1,65.0,toyota,gas,std,four,sedan,fwd,front,102.4,175.6,66.5,54.9,2326,ohc,four,122,mpfi,3.31,3.54,8.7,92.0,4200.0,29,34,8948.0
174,-1,65.0,toyota,diesel,turbo,four,sedan,fwd,front,102.4,175.6,66.5,54.9,2480,ohc,four,110,idi,3.27,3.35,22.5,73.0,4500.0,30,33,10698.0
176,-1,65.0,toyota,gas,std,four,sedan,fwd,front,102.4,175.6,66.5,54.9,2414,ohc,four,122,mpfi,3.31,3.54,8.7,92.0,4200.0,27,32,10898.0
180,-1,90.0,toyota,gas,std,four,sedan,rwd,front,104.5,187.8,66.5,54.1,3131,dohc,six,171,mpfi,3.27,3.35,9.2,156.0,5200.0,20,24,15690.0


##3. Aggregation with the `agg()` method

Once the GroupBy object has been created, several methods are available to perform a computation on the grouped data. 
The `aggregate()` or equivalently `agg()` method is the most general way to summarize grouped dataframes. 
The following cells in this section demonstrate these methods.

Apply the `np.mean` function to the dataframe with the `make`, `body_style`, `city_mpg` and `highway_mpg` columns grouped by the `make` and `body_style` columns.

In [10]:
import_df \
  .loc[:,['make','body_style','city_mpg','highway_mpg']] \
  .groupby(['make','body_style']) \
  .agg(np.mean)

city_mpg  highway_mpg
make          body_style                         
alfa-romero   convertible  21.000000    27.000000
              hatchback    19.000000    26.000000
audi          hatchback    16.000000    22.000000
              sedan        19.400000    24.400000
              wagon        19.000000    25.000000
bmw           sedan        19.375000    25.375000
chevrolet     hatchback    42.500000    48.000000
              sedan        38.000000    43.000000
dodge         hatchback    28.400000    34.200000
              sedan        28.666667    35.333333
              wagon        24.000000    30.000000
honda         hatchback    33.142857    38.285714
              sedan        26.600000    31.800000
              wagon        30.000000    34.000000
isuzu         hatchback    24.000000    29.000000
              sedan        33.333333    38.333333
jaguar        sedan        14.333333    18.333333
mazda         hatchback    23.700000    29.500000
              sedan        28.571429    35.428571
mercedes-benz convertible  16.000000    18.000000
              hardtop      18.000000    20.500000
              sedan        18.500000    21.000000
              wagon        22.000000    25.000000
mercury       hatchback    19.000000    24.000000
mitsubishi    hatchback    25.333333    31.222222
              sedan        24.000000    31.000000
nissan        hardtop      31.000000    37.000000
              hatchback    22.600000    28.800000
              sedan        29.222222    35.111111
              wagon        26.333333    32.000000
peugot        sedan        22.714286    27.857143
              wagon        22.000000    24.500000
plymouth      hatchback    27.750000    33.250000
              sedan        31.000000    38.000000
              wagon        24.000000    30.000000
porsche       convertible  17.000000    25.000000
              hardtop      17.000000    25.000000
              hatchback    18.000000    27.500000
renault       hatchback    23.000000    31.000000
              wagon        23.000000    31.000000
saab          hatchback    20.333333    27.333333
              sedan        20.333333    27.333333
subaru        hatchback    27.666667    32.666667
              sedan        26.800000    31.200000
              wagon        24.750000    28.750000
toyota        convertible  24.000000    30.000000
              hardtop      24.000000    30.000000
              hatchback    27.785714    33.428571
              sedan        29.100000    34.000000
              wagon        26.000000    31.250000
volkswagen    convertible  24.000000    29.000000
              hatchback    24.000000    29.000000
              sedan        30.000000    36.666667
              wagon        25.000000    31.000000
volvo         sedan        21.125000    25.750000
              wagon        21.333333    26.000000

Apply multiple functions to all non-grouping columns by passing a list of functions to the `agg` or `aggregate` method.

In [11]:
import_df \
  .loc[:,['make','body_style','city_mpg','highway_mpg']] \
  .groupby(['make','body_style']) \
  .agg([np.mean, np.max, np.min]) \
  .head()

city_mpg           highway_mpg          
                            mean amax amin        mean amax amin
make        body_style                                          
alfa-romero convertible     21.0   21   21        27.0   27   27
            hatchback       19.0   19   19        26.0   26   26
audi        hatchback       16.0   16   16        22.0   22   22
            sedan           19.4   24   17        24.4   30   20
            wagon           19.0   19   19        25.0   25   25

Apply specific functions to specific dataframe columns by passing a dict to the `agg` or `aggregate` method.

In [12]:
import_df \
  .loc[:,['make','body_style','city_mpg','highway_mpg']] \
  .groupby(['make','body_style']) \
  .agg({'city_mpg'   : 'mean',
        'highway_mpg': lambda x: np.mean(x),
         }) \
  .head()

city_mpg  highway_mpg
make        body_style                        
alfa-romero convertible      21.0         27.0
            hatchback        19.0         26.0
audi        hatchback        16.0         22.0
            sedan            19.4         24.4
            wagon            19.0         25.0

Notice that the summary function can also be specified as a string and using an anonymous function.

To name output columns use a nested dict (as in the following example). 
- Keys of the outer dictionary name existing columns in the grouped dataframe
- Keys of the inner dictionary name the summary columns of the resulting dataframe

Values of the inner dictionary can be strings, numpy functions or lamda functions.

In [13]:
import_df.columns

Index(['symboling', 'normalized_losses', 'make', 'fuel_type', 'aspiration',
       'num_of_doors', 'body_style', 'drive_wheels', 'engine_location',
       'wheel_base', 'length', 'width', 'height', 'curb_weight', 'engine_type',
       'num_of_cylinders', 'engine_size', 'fuel_system', 'bore', 'stroke',
       'compression_ratio', 'horsepower', 'peak_rpm', 'city_mpg',
       'highway_mpg', 'price'],
      dtype='object')

In [14]:
import_df \
  .groupby(['engine_type']) \
  .agg({'width'   : {'mean': np.mean,
                     'min' : np.min,
                     'max' : np.max
                    },
        'length'  : {'mean': 'mean',
                     'min' : np.min,
                     'max' : lambda x: np.max(x)}
       })

/usr/local/lib/python3.6/dist-packages/pandas/core/groupby/generic.py:1315: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


width                  length              
                  mean   min   max        mean    min    max
engine_type                                                 
dohc         66.400000  64.0  69.6  182.500000  168.7  199.6
dohcv        72.300000  72.3  72.3  175.700000  175.7  175.7
l            67.716667  60.3  68.4  186.966667  141.1  198.9
ohc          65.527703  61.8  71.7  171.953378  144.6  202.6
ohcf         64.960000  63.4  65.4  168.866667  156.9  173.6
ohcv         68.776923  65.5  72.0  185.592308  170.7  208.1
rotor        65.700000  65.7  65.7  169.000000  169.0  169.0

##4. Other aggregating functions

There are other aggregation functions are included as GroupBy methods. Some commonly used aggregating functions are:
- `mean()`	Compute mean of groups
- `sum()`	Compute sum of group values
- `size()`	Compute group sizes
- `std()`	Standard deviation of groups
- `sem()`	Standard error of the mean of groups
- `describe()`	Generates descriptive statistics
- `min()`	Compute min of group values
- `max()`	Compute max of group values

The following cells in this section are some examples showing the use of those aggregation functions.

In each case the summary function is applied to all non-grouping columns of the grouped dataframe.

Display the mean of `city_mpg` and `highway_mpg` for each kind of `make`.

In [15]:
import_df[['make','city_mpg','highway_mpg']].groupby('make').mean()

,city_mpg,highway_mpg
make,,
alfa-romero,20.333333,26.666667
audi,18.857143,24.142857
bmw,19.375000,25.375000
chevrolet,41.000000,46.333333
dodge,28.000000,34.111111
honda,30.384615,35.461538
isuzu,31.000000,36.000000
jaguar,14.333333,18.333333
mazda,25.705882,31.941176


Display the mean of `city_mpg` and `highway_mpg` for each unique combination of `make` and `body_style`. Sort the values according to the  `city_mpg` in a descending way.

In [16]:
import_df[['make','body_style','city_mpg','highway_mpg']] \
  .groupby(['make','body_style'])\
  .mean()\
  .sort_values(by='city_mpg',ascending=False)

city_mpg  highway_mpg
make          body_style                         
chevrolet     hatchback    42.500000    48.000000
              sedan        38.000000    43.000000
isuzu         sedan        33.333333    38.333333
honda         hatchback    33.142857    38.285714
plymouth      sedan        31.000000    38.000000
nissan        hardtop      31.000000    37.000000
volkswagen    sedan        30.000000    36.666667
honda         wagon        30.000000    34.000000
nissan        sedan        29.222222    35.111111
toyota        sedan        29.100000    34.000000
dodge         sedan        28.666667    35.333333
mazda         sedan        28.571429    35.428571
dodge         hatchback    28.400000    34.200000
toyota        hatchback    27.785714    33.428571
plymouth      hatchback    27.750000    33.250000
subaru        hatchback    27.666667    32.666667
              sedan        26.800000    31.200000
honda         sedan        26.600000    31.800000
nissan        wagon        26.333333    32.000000
toyota        wagon        26.000000    31.250000
mitsubishi    hatchback    25.333333    31.222222
volkswagen    wagon        25.000000    31.000000
subaru        wagon        24.750000    28.750000
volkswagen    hatchback    24.000000    29.000000
plymouth      wagon        24.000000    30.000000
dodge         wagon        24.000000    30.000000
mitsubishi    sedan        24.000000    31.000000
toyota        convertible  24.000000    30.000000
              hardtop      24.000000    30.000000
volkswagen    convertible  24.000000    29.000000
isuzu         hatchback    24.000000    29.000000
mazda         hatchback    23.700000    29.500000
renault       hatchback    23.000000    31.000000
              wagon        23.000000    31.000000
peugot        sedan        22.714286    27.857143
nissan        hatchback    22.600000    28.800000
mercedes-benz wagon        22.000000    25.000000
peugot        wagon        22.000000    24.500000
volvo         wagon        21.333333    26.000000
              sedan        21.125000    25.750000
alfa-romero   convertible  21.000000    27.000000
saab          sedan        20.333333    27.333333
              hatchback    20.333333    27.333333
audi          sedan        19.400000    24.400000
bmw           sedan        19.375000    25.375000
alfa-romero   hatchback    19.000000    26.000000
mercury       hatchback    19.000000    24.000000
audi          wagon        19.000000    25.000000
mercedes-benz sedan        18.500000    21.000000
porsche       hatchback    18.000000    27.500000
mercedes-benz hardtop      18.000000    20.500000
porsche       convertible  17.000000    25.000000
              hardtop      17.000000    25.000000
mercedes-benz convertible  16.000000    18.000000
audi          hatchback    16.000000    22.000000
jaguar        sedan        14.333333    18.333333

Display the number of records in each unique combination of `make` and `body_style` in the dataframe.

In [17]:
import_df[['make','body_style','city_mpg','highway_mpg']]\
       .groupby(['make','body_style'])\
       .size()

make           body_style 
alfa-romero    convertible     2
               hatchback       1
audi           hatchback       1
               sedan           5
               wagon           1
bmw            sedan           8
chevrolet      hatchback       2
               sedan           1
dodge          hatchback       5
               sedan           3
               wagon           1
honda          hatchback       7
               sedan           5
               wagon           1
isuzu          hatchback       1
               sedan           3
jaguar         sedan           3
mazda          hatchback      10
               sedan           7
mercedes-benz  convertible     1
               hardtop         2
               sedan           4
               wagon           1
mercury        hatchback       1
mitsubishi     hatchback       9
               sedan           4
nissan         hardtop         1
               hatchback       5
               sedan           9
               w

Display the descriptive statistics of `city_mpg` and `highway_mpg` for every value of the `make` column.

In [18]:
import_df[['make','city_mpg','highway_mpg']]\
  .groupby(['make'])\
  .describe()

city_mpg                             ... highway_mpg                   
                 count       mean       std   min  ...         25%   50%    75%   max
make                                               ...                               
alfa-romero        3.0  20.333333  1.154701  19.0  ...        26.5  27.0  27.00  27.0
audi               7.0  18.857143  2.544836  16.0  ...        22.0  25.0  25.00  30.0
bmw                8.0  19.375000  3.248626  15.0  ...        22.0  26.5  28.25  29.0
chevrolet          3.0  41.000000  5.196152  38.0  ...        43.0  43.0  48.00  53.0
dodge              9.0  28.000000  5.545268  19.0  ...        30.0  38.0  38.00  41.0
honda             13.0  30.384615  6.589619  24.0  ...        33.0  34.0  34.00  54.0
isuzu              4.0  31.000000  8.082904  24.0  ...        29.0  36.0  43.00  43.0
jaguar             3.0  14.333333  1.154701  13.0  ...        18.0  19.0  19.00  19.0
mazda             17.0  25.705882  6.282562  16.0  ...        27.0  32.0  38.00  42.0
mercedes-benz      8.0  18.500000  3.817254  14.0  ...        17.5  21.5  25.00  25.0
mercury            1.0  19.000000       NaN  19.0  ...        24.0  24.0  24.00  24.0
mitsubishi        13.0  24.923077  5.298766  19.0  ...        30.0  30.0  32.00  41.0
nissan            18.0  27.000000  7.514691  17.0  ...        25.0  37.0  37.00  50.0
peugot            11.0  22.454545  4.297991  18.0  ...        24.0  24.0  29.00  33.0
plymouth           7.0  28.142857  6.067085  19.0  ...        30.0  38.0  38.00  41.0
porsche            5.0  17.400000  0.894427  17.0  ...        25.0  25.0  27.00  28.0
renault            2.0  23.000000  0.000000  23.0  ...        31.0  31.0  31.00  31.0
saab               6.0  20.333333  1.032796  19.0  ...        26.5  28.0  28.00  28.0
subaru            12.0  26.333333  2.933609  23.0  ...        29.0  31.0  32.25  37.0
toyota            32.0  27.500000  4.812618  19.0  ...        30.0  32.0  36.25  47.0
volkswagen        12.0  28.583333  5.976596  19.0  ...        30.5  34.0  39.00  46.0
volvo             11.0  21.181818  3.219260  17.0  ...        24.0  27.0  28.00  28.0

[22 rows x 16 columns]

__The End__